In [97]:
import urllib3
import re
import requests
from bs4 import BeautifulSoup
from pprint import pprint
from statistics import mean

## Получение всех правок

In [106]:
def GetRevisions(pageTitle):
    url = "https://en.wikipedia.org/w/api.php?action=query&format=xml&prop=revisions&rvlimit=500&titles=" + pageTitle
    revisions = [] # Правки                                        
    next = '' # параметр для следующего URL - определяется по ИД последней просмотренной правки                                            
    while True:
        response = requests.get(url + next)
        revisions += re.findall('<rev [^>]*>', str(response.content)) # добавление всех правок в список
        cont = re.search('<continue rvcontinue="([^"]+)"', str(response.content))
        if not cont: # если переход далее невозможен, то останавливаем                                      
            break
        next = "&rvcontinue=" + cont.group(1) # определяем ID на котором окончился поиск
    return revisions;

In [107]:
revs = GetRevisions("Python_(programming_language)")

### Получение всех IP-адресов

In [108]:
ips = []
for elem in revs:
    ips += re.findall(r'\"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\"', elem)

In [53]:
ips_sample = ips[:250]

In [55]:
countries = dict()

## Составление словаря (Страна, кол-во вхождений)

In [70]:
for elem in ips_sample:
    url = "http://api.ipstack.com/"+elem.replace(r'"', '')+"?access_key=f173866b81b8388a37abc474bae4f1b8"
    request = requests.get(url)
    obj = request.json()
    if countries.get(obj["country_name"]) == None:
        countries.update({obj["country_name"] : 1})
    else:
        countries[obj["country_name"]] = countries[obj["country_name"]] + 1

In [71]:
countries

{'United States': 39,
 'India': 72,
 'United Kingdom': 33,
 'Israel': 1,
 'Latvia': 1,
 'Egypt': 1,
 'Canada': 6,
 'Pakistan': 1,
 'Iran': 1,
 'Spain': 3,
 'Croatia': 2,
 'Australia': 7,
 'Myanmar (Burma)': 1,
 'Sweden': 35,
 'Argentina': 9,
 'Rwanda': 1,
 'Germany': 2,
 'Belgium': 6,
 'Taiwan': 4,
 'United Arab Emirates': 3,
 'Switzerland': 1,
 'Congo - Kinshasa': 1,
 'Malawi': 1,
 'Lebanon': 1,
 'Turkey': 2,
 'Bolivia': 1,
 'Singapore': 1,
 'Morocco': 3,
 'Russia': 6,
 'Greece': 1,
 'Hong Kong SAR China': 1,
 'France': 1,
 'Indonesia': 3,
 'Philippines': 1}

### Сортировка словаря и вывод стран по убыванию

In [73]:
for k in sorted(countries, key=countries.get, reverse=True):
    print(k + " " + str(countries[k]))

India 72
United States 39
Sweden 35
United Kingdom 33
Argentina 9
Australia 7
Canada 6
Belgium 6
Russia 6
Taiwan 4
Spain 3
United Arab Emirates 3
Morocco 3
Indonesia 3
Croatia 2
Germany 2
Turkey 2
Israel 1
Latvia 1
Egypt 1
Pakistan 1
Iran 1
Myanmar (Burma) 1
Rwanda 1
Switzerland 1
Congo - Kinshasa 1
Malawi 1
Lebanon 1
Bolivia 1
Singapore 1
Greece 1
Hong Kong SAR China 1
France 1
Philippines 1


### Подсчет средней з/п по предложению

In [111]:
def GetSalaries():
    url = 'https://api.hh.ru/vacancies?industry=7.540&currency=RUR&text=Python&per_page=10&page=0'
    request = requests.get(url)
    pages = request.json().get('pages')
    salaries = []
    for i in range(pages):
        url = 'https://api.hh.ru/vacancies?industry=7.540&currency=RUR&text=Python&per_page=10&page='+ str(i)
        request = requests.get(url)
        page = request.json()
        for item in page.get('items'):
            if item['salary'] != None:
                salary = 0
                times = 0
                if item['salary']['from'] != None:
                    salary += int(item['salary']['from'])
                    times += 1
                if item['salary']['to'] != None:
                    salary += int(item['salary']['to'])
                    times += 1
                if times > 0:
                    salaries.append(salary / times)
    return salaries

## Средняя з/п:

In [112]:
mean(GetSalaries())

117518.62903225806